# Build Treasury Quotes CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')

In [4]:
DATE = '2024-12-31'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204085.0,20250815.106870,2024-12-31,101.843702,101.856069,101.849886,M,C,2.578125,-0.000159,0.000105,221.070587,3356.0,11187.0,0.0,NaN,None
1,204089.0,20270215.106620,2024-12-31,105.188744,105.313744,105.251244,M,C,2.484375,0.000505,0.000109,721.164952,2856.0,9522.0,0.0,NaN,None
2,204094.0,20290215.105250,2024-12-31,103.654038,103.700913,103.677475,M,C,1.968750,-0.000477,0.000116,1351.975671,4278.0,11350.0,0.0,NaN,None
3,206142.0,20400515.104370,2024-12-31,95.660066,95.722566,95.691316,M,C,0.555939,-0.002951,0.000129,4083.212693,13042.0,43473.0,0.0,NaN,None
4,206468.0,20250115.B02370,2024-12-31,99.941117,99.941830,99.941473,M,C,0.000000,NaN,NaN,NaN,16286.0,46890.0,0.0,1.67458,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,208328.0,20250620.400000,2024-12-31,98.064797,98.069492,98.067145,M,C,0.000000,0.000349,0.000114,171.000000,NaN,72275.0,0.0,NaN,None
446,208329.0,20250626.400000,2024-12-31,97.994875,97.999736,97.997306,M,C,0.000000,0.000335,0.000114,177.000000,NaN,73409.0,0.0,NaN,None
447,208330.0,20250703.400000,2024-12-31,97.915039,97.920095,97.917567,M,C,0.000000,0.000063,0.000114,184.000000,NaN,NaN,0.0,NaN,None
448,208331.0,20251226.400000,2024-12-31,96.049507,96.066910,96.058208,M,C,0.000000,0.000346,0.000112,360.000000,NaN,48938.0,0.0,NaN,None


In [5]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204084.0,20250215.107620,20250215.107620,912810ET,1995-02-15,2025-02-15,0.0,7.625,2.0,3.8125,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204085.0,20250815.106870,20250815.106870,912810EV,1995-08-15,2025-08-15,0.0,6.875,2.0,3.4375,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204086.0,20260215.106000,20260215.106000,912810EW,1996-02-15,2026-02-15,0.0,6.000,2.0,3.0000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204087.0,20260815.106750,20260815.106750,912810EX,1996-08-15,2026-08-15,0.0,6.750,2.0,3.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204088.0,20261115.106500,20261115.106500,912810EY,1996-11-15,2026-11-15,0.0,6.500,2.0,3.2500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,208331.0,20251226.400000,20251226.400000,912797NU,2024-12-26,2025-12-26,0.0,0.000,0.0,0.0000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
446,208332.0,20261231.204250,20261231.204250,91282CME,2024-12-31,2026-12-31,0.0,4.250,2.0,2.1250,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
447,208333.0,20271215.204000,20271215.204000,91282CMB,2024-12-15,2027-12-15,0.0,4.000,2.0,2.0000,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
448,208334.0,20291231.204370,20291231.204370,91282CMD,2024-12-31,2029-12-31,0.0,4.375,2.0,2.1875,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


In [6]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')
rawdata

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204085.0,20250815.106870,2024-12-31,101.843702,101.856069,101.849886,M,C,2.578125,-0.000159,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204089.0,20270215.106620,2024-12-31,105.188744,105.313744,105.251244,M,C,2.484375,0.000505,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204094.0,20290215.105250,2024-12-31,103.654038,103.700913,103.677475,M,C,1.968750,-0.000477,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,206142.0,20400515.104370,2024-12-31,95.660066,95.722566,95.691316,M,C,0.555939,-0.002951,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,206468.0,20250115.B02370,2024-12-31,99.941117,99.941830,99.941473,M,C,0.000000,NaN,...,None,0.0,None,11.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,208328.0,20250620.400000,2024-12-31,98.064797,98.069492,98.067145,M,C,0.000000,0.000349,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
446,208329.0,20250626.400000,2024-12-31,97.994875,97.999736,97.997306,M,C,0.000000,0.000335,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
447,208330.0,20250703.400000,2024-12-31,97.915039,97.920095,97.917567,M,C,0.000000,0.000063,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
448,208331.0,20251226.400000,2024-12-31,96.049507,96.066910,96.058208,M,C,0.000000,0.000346,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


***

# Process Data

In [7]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data, select_maturities

data = process_wrds_treasury_data(rawdata)

In [8]:
idx = select_maturities(data)
data_select = data.loc[idx]

***

# Save to Excel

In [9]:
### raw data
outfile = f'../data/treasury_quotes_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    rawdata.columns = rawdata.columns.str.upper()
    rawdata.to_excel(writer, sheet_name= 'quotes', index=False)


### processed data
outfile = f'../data/treasury_quotes_clean_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=True)
    data_select.to_excel(writer, sheet_name = 'selected quotes', index=True)